In [1]:
import ee
import geemap
import os
import csv

# Authenticate and initialize Earth Engine
try:
    ee.Initialize()
except Exception as e:
    print(f"Error during Earth Engine Initialization: {e}")

# Load the shapefile
shapefile = ee.FeatureCollection('projects/ee-hayatabid/assets/Jehlum_Chenab')

# Function to apply scaling factors
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(
        thermal_bands, None, True
    )

# Load Landsat 8 imagery, filter by location and date, then apply scaling factors
image_collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                    .filterBounds(shapefile)
                    .filterDate('2021-05-01', '2021-08-01')
                    .map(apply_scale_factors))

# Create a median mosaic of the image collection
mosaic = image_collection.median()

# Clip the mosaic to the boundaries of the shapefile
clipped_mosaic = mosaic.clip(shapefile.geometry().bounds())

# Visualization parameters
vis_params = {'min': 0.0, 'max': 0.3, 'bands': ['SR_B4', 'SR_B3', 'SR_B2']}

# Display the result on the map
Map = geemap.Map()
Map.centerObject(shapefile, 8)
Map.addLayer(clipped_mosaic, vis_params, "Landsat-8 Mosaic")
Map.addLayer(shapefile, {}, 'Shapefile')

# Load ESA WorldCover data and add it to the map
esa = ee.ImageCollection('ESA/WorldCover/v200').first().clip(shapefile.geometry().bounds())
Map.addLayer(esa, {}, 'ESA')

# Create a training dataset from ESA WorldCover data
points = esa.sample(
    **{
        'region': shapefile.geometry().bounds(),
        'scale': 3,
        'numPixels': 50000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)
Map.addLayer(points, {}, 'training', False)

# Use these bands for prediction
bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']

# This property of the table stores the land cover labels
label = 'Map'

# Overlay the points on the imagery to get training
training_points = clipped_mosaic.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 30}
)

# Split the dataset into training and validation sets
sample = training_points.randomColumn('random', 0)
split = 0.7
training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

# Train a CART classifier with default parameters
classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

# Classify the image with the same bands used for training
result = clipped_mosaic.select(bands).classify(classifier)

# Display the clusters with random colors
Map.addLayer(result.randomVisualizer(), {}, 'classified')

# Adds ESA WorldCover legend to the map
class_values = esa.get('Map_class_values').getInfo()
class_palette = esa.get('Map_class_palette').getInfo()

landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

Map.addLayer(landcover, {}, 'Land cover')
Map.add_legend(builtin_legend='ESA_WorldCover')

# Calculate training accuracy
train_accuracy = classifier.confusionMatrix()
print('Training accuracy:')
print('Confusion matrix:\n', train_accuracy.getInfo())
print('Overall accuracy:', train_accuracy.accuracy().getInfo())
print('Kappa:', train_accuracy.kappa().getInfo())
print('Producer\'s accuracy:', train_accuracy.producersAccuracy().getInfo())
print('Consumer\'s accuracy:', train_accuracy.consumersAccuracy().getInfo())

# Validate the classifier
validated = validation.classify(classifier)

# Calculate test accuracy
test_accuracy = validated.errorMatrix(label, 'classification')
print('Test accuracy:')
print('Confusion matrix:\n', test_accuracy.getInfo())
print('Overall accuracy:', test_accuracy.accuracy().getInfo())
print('Kappa:', test_accuracy.kappa().getInfo())
print('Producer\'s accuracy:', test_accuracy.producersAccuracy().getInfo())
print('Consumer\'s accuracy:', test_accuracy.consumersAccuracy().getInfo())

# Export accuracy matrices to CSV
out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
training_csv = os.path.join(out_dir, "train_accuracy.csv")
testing_csv = os.path.join(out_dir, "test_accuracy.csv")

with open(training_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(train_accuracy.getInfo())

with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(test_accuracy.getInfo())

# Add final land cover classification layer to the map
Map.addLayer(landcover, {}, "Final land cover")
Map
